# Problema de las ocho reinas

El problema consiste en poner en un tablero de 8x8 a ocho reinas sin que ninguna logre comerse entre ellas. 
La manera en que será realizado este problema será por medio de algoritmos evolutivos. Primero construiremos un individuo el cual será probado y mejorado hasta llegar a la solución. 
Para esto primero hacemos el tablero, lo hacemos en un archivo csv y después lo importamos a python para generar una matriz de 8x8.
La matriz contendrá 0’s por default y los 1’s serán las reinas.

![title](https://www.researchgate.net/profile/Ellips_Masehian/publication/278681097/figure/fig1/AS:391891729305602@1470445677335/a-A-solution-to-the-non-attacking-8-queens-problem-b-An-optimal-solution-to-the.png)

In [43]:
import pandas 
import numpy as np
df = pandas.read_csv(r"C:\Users\wachu\Documents\IIA\Semestre 6\Optimizacion & Metaheuristicas II\P1\programas\ochoR.csv",header = None)
tablero = df.values
tablero

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

### Creación del individuo y población

Hacemos una lista que contenga números del 1 al 8, estos números serán las posiciones de nuestras reinas, una vez echa la lista hacemos una permutación, para que todas tengan posiciones diferentes. De esta manera es menos probable que haya reinas en posición vertical.
Una vez hecha la función, hacemos otra función que nos servirá para hacer la población. Lo único que hará es llamar la función “create_individuo” N veces, donde N sea el numero de individuos que queramos en nuestra población.

In [44]:
def create_individuo():
    individuo = [i for i in range(8)]
    np.random.shuffle(individuo)
    return individuo
    
def create_population(NumP):
    population = []
    for i in range(NumP):
        population.append(create_individuo())
    return population


### Aptitud (fitness)

Para calcular el fitness lo primero que tenemos que hacer es poner las reinas en el tablero. Para esto, simplemente iteramos al individuo y ponemos los números en diferentes columnas. 

In [45]:
def fenotipo(ind):
    fenoMat = np.copy(tablero)
    for i in range(8):
        fenoMat[i,ind[i]] = 1
    return fenoMat

Una vez obtenida la matriz entonces, checamos primero el movimiento vertical de las reinas. Sabiendo que solo debe de estar una reina en cada columna diferente y que el tablero esta echo de ceros, sumamos columna por columna, si la suma de los números en las columnas es mayor a 1 entonces quiere decir que hay un error y lo añadimos a nuestro contador llamado “fit”, si no entonces no hay ningún error en la parte de las columnas.
Debido a la manera en que se generan los individuos, nunca habrá errores al checar las columnas. Lo cual hace más optimo la resolución del problema. Básicamente podemos omitir este paso y continuar con las diagonales

In [46]:
def fitnessIndColumnas(matIndv):
    fit = 0
    lenMat = len(matIndv)-1
    for i in range(8):
        if np.sum(matIndv[:,i]) > 1:
            fit+=1

Ahora falta checar el movimiento en diagonal de las reinas. Lo que hacemos es: 
Primero detectar a la reina, para esto sacamos el “argmax” esto nos da la posición de el numero mas grande de la fila, que en este caso siempre será 1, el cual será la reina. Una vez obtenida la posición de la reina iteramos en dos while’s.



 Para checar la diagonal derecha, aumentamos la posición en horizontal y vertical para así llegar a la posición diagonal. Primero preguntamos si la posición en vertical (columna) es menor al tamaño del arreglo en vertical. Hacemos esto mientras nuestra posición en horizontal sea menor al tamaño de nuestro arreglo, esto es que no se pase del tamaño del lado derecho.
 

Hacemos lo mismo para la posición izquierda, solo que, en vez de aumentar la posición en horizontal, la disminuimos para así llegar a la posición diagonal. Hacemos esto mientras nuestra posición en la fila sea mayor a 0, esto es que no se pase del tamaño, pero ahora del lado izquierdo
Cada que encuentre un 1 aumentamos nuestro contador “fit”.

In [47]:
def fitnessInd(matIndv):
    fit = 0
    lenMat = len(matIndv)-1
    for i in range(8):
        if np.sum(matIndv[:,i]) > 1:
            fit+=1
    j=indx = indx2 = clmn2 = 0
    for x in range(8):
        clmn = clmn2 = j 
        m = matIndv[x]
        indx = np.argmax(matIndv[x])
        indx2 = indx
        while indx < lenMat:
            if clmn < lenMat:
                indx+=1
                clmn+=1
                if matIndv[clmn,indx] == 1: fit+=1
            else: break
        while indx2 >0:
            if clmn2 < lenMat:
                indx2-=1
                clmn2+=1
                if matIndv[clmn2,indx2] == 1: fit+=1
            else: break
        j+=1

    return fit

Finalmente hacemos una función que llame a “fitnessInd” N veces, donde N sea el numero de individuos que queramos en nuestra población.

In [48]:
def fitness(inds):
    fitnessPopu = []
    for i in inds:
        fenoInd = fenotipo(i)
        fitnessPopu.append(fitnessInd(fenoInd))
    return fitnessPopu

### Reproduccion

Para hacer la reproducción lo que hice fue generar un numero random de la mitad del tamaño del individuo (8) esto es para seleccionar a los hijos del padre 1, después los añadimos a un nuevo arreglo que se va a llamar offspring. Una vez que el offspring tenga los elementos del primer padre checamos cuales que no estén en el padre 2 no están en el offspring y los agregamos a este.

In [49]:
def crossoverInds(ind1,ind2):
    offspring=[]
    limit = np.random.randint(4)
    for i in range(0,limit):
        offspring.append(ind1[i])

    for j in ind2:
        if j not in offspring:
            offspring.append(j)
    return offspring

Finalmente hacemos una función que llame a “crossover” en la que agarramos dos individuos random de la población y generamos un número aleatorio de 0 a 1, después  a estos les aplicamos la función crossoverInds siempre y cuando la probabilidad que sacamos sea menor a la probabilidad de cruza (0.7), si no es menor, entonces agregamos al padre tal cual

In [50]:
def crossover(popu,Pc,NumP):
    offspringPopu = []
    while len(offspringPopu) < NumP:
        ProC = np.random.random(1)
        n1Rand = np.random.randint(NumP)
        n2Rand = np.random.randint(NumP)
        if ProC < Pc:
            offspringPopu.append(crossoverInds(popu[n1Rand],popu[n2Rand]))
        else:
            offspringPopu.append(popu[n1Rand])
    return offspringPopu

### Mutación

Para la mutación simplemente hacemos un shuffle de todos los datos para cambiarlos todos de posición. Posteriormente hacemos una función que mande llamar a esa función siempre y cuando nuestra probabilidad de mutación (0.3) sea mayor a la probabilidad que saquemos aleatoriamente. Si no es, agregamos el individuo tal cual.

In [51]:
def mutacionInd(ind):
    np.random.shuffle(ind)
    return ind

def mutacion(popu,Pm,NumP):
    mutacionPopu = []
    proM = 0
    for ind in popu:
        proM = np.random.random(1)
        if proM < Pm:
            mutacionPopu.append(mutacionInd(ind))
        else:
            mutacionPopu.append(ind)
    return mutacionPopu

### Elite

Para sacar el individuo elite lo que se hace es sacar el index del valor más bajo en el fitness, ese index nos servirá para buscar al individuo en la población, una vez obtenido regresamos el individuo y el fitness

In [52]:
def elite(popu,fit):
    indexE = np.argmin(fitnes)
    minFit = np.amin(fitnes)
    return popu[indexE],minFit

Para seleccionar el individuo elite nuevo, lo que hacemos es similar a obtener el individuo elite, esto es sacar el index del valor más bajo en el fitness para luego buscarlo en la populación, así mismo sacamos el valor mínimo del fitness y lo comparamos con el que teníamos, si es menor cambiamos al elite si no, lo añadimos a la población para variar

In [56]:
def selectElite(popu,fit,currentElite,currentFit,NumP):
    index = np.argmin(fit)
    minFit = np.amin(fit)
    if minFit < currentFit:
        currentFit = minFit
        currentElite = popu[index]
    else:
        n = np.random.randint(NumP)
        popu[n] = currentElite
        fit[n] = currentFit
    return popu,fit,currentElite,currentFit

Para la selección lo que hacemos es también sacar dos individuos aleatorios y calcular el fitness mínimo de los dos y ese es el que metemos al arreglo 

In [57]:
def selection(popu,fit,NumP):
    selectArray = []
    while len(selectArray)<NumP:
        n1 = np.random.randint(NumP)
        n2 = np.random.randint(NumP)
        minVal = min(fit[n1],fit[n2])
        selectArray.append(popu[fit.index(minVal)])
    return selectArray

## Algoritmo Genetico

Una vez que sacamos todas las funciones entonces empezamos a implementarlas, primero sacamos nuestra población inicial, fitness y nuestro primer individuo elite, en este caso nuestra población va a ser de 100 elementos, nuestra probabilidad de cruza de 0.7 y la de mutación de 0.3, nuestra generación será de 100.
La generación es las veces que se van a repetir nuestros pasos en el algoritmo, o hasta que encuentre el individuo con fitness 0, es decir la respuesta a nuestro problema.
Mientras no pase esto lo que hacemos es hacer la selección, reproducción y mutación de nuestra población, una vez que la sacamos vemos si hay un nuevo elemento elite, y así lo hacemos G veces donde G es el numero de generaciones. 
Imprimimos en este caso en cada generación el fitness y el num de generación, es en cada generación por que el algoritmo no tarda tanto en encontrar la solución.


In [61]:
N = 100
Pc = 0.7
Pm = 0.3
G = 1000

population = create_population(N)
fitnes = fitness(population)
ind_elit,fit_elite = elite(population,fitnes)
g=0

while fit_elite > 0 and g < G:
    population = selection(population,fitnes,N)
    population = crossover(population,Pc,N)
    population = mutacion(population,Pm,N)
    fitnes = fitness(population)
    population,fitnes,ind_elit,fit_elite = selectElite(population,fitnes,ind_elit,fit_elite,N)
    g+=1
    if g%1==0:
        print('Generation:',g,' fitness:',fit_elite)
    print(ind_elit)
print(fenotipo(ind_elit))

Generation: 1  fitness: 1
[7, 5, 1, 4, 3, 2, 0, 6]
Generation: 2  fitness: 1
[2, 4, 5, 0, 6, 3, 1, 7]
Generation: 3  fitness: 1
[0, 4, 1, 7, 2, 6, 3, 5]
Generation: 4  fitness: 1
[0, 4, 1, 7, 2, 6, 3, 5]
Generation: 5  fitness: 1
[0, 4, 1, 7, 2, 6, 3, 5]
Generation: 6  fitness: 1
[3, 7, 1, 0, 5, 6, 4, 2]
Generation: 7  fitness: 1
[3, 7, 1, 0, 5, 6, 4, 2]
Generation: 8  fitness: 1
[3, 7, 1, 0, 5, 6, 4, 2]
Generation: 9  fitness: 1
[3, 6, 5, 2, 0, 7, 4, 1]
Generation: 10  fitness: 1
[3, 6, 5, 2, 0, 7, 4, 1]
Generation: 11  fitness: 1
[3, 6, 5, 2, 0, 7, 4, 1]
Generation: 12  fitness: 1
[3, 6, 5, 2, 0, 7, 4, 1]
Generation: 13  fitness: 0
[4, 7, 3, 0, 6, 1, 5, 2]
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]]
